In [1]:
# Khai báo các thư viện cần thiết
from new_import import *

# Khai báo đường dẫn đến kết quả phân loại và kết quả thực tế của địa phương
KD_path = "ThuanHoa/KhoanhDat/ThuanHoa_TKDD2022.shp"
KetQuaPhanLoaiDat = "KetQuaPhanLoaiDat.tif"

In [2]:
# khai báo các loại đất ứng với các mã đất phân loại được
CODE_MAP = {
    "BHK": 2,
    "CLN": 3,
    "DGD": 6,
    "DGT": 6,
    "DNL": 6,
    "DRA": 6,
    "DSH": 6,
    "DTL": 5,
    "DTS": 6,
    "DYT": 6,
    "LUC": 1,
    "NKH": 3,
    "NTD": 6,
    "NTS": 4,
    "ONT": 6,
    "SKC": 6,
    "SKX": 6,
    "SON": 5,
    "TMD": 6,
    "TON": 6,
    "TSC": 6,
}

# Khai báo các nhãn phân loại đất ứng với 3 loại đất chính
HT_MAP = {
    "NN": {"name": "Đất Nông Nghiệp", "data": [1, 2, 3, 4]},
    "PNN": {"name": "Đất Phi Nông Nghiệp", "data": [6]},
    "TQ": {"name": "Đất Thổ Quả", "data": [15]},
}

In [ ]:
# Tiến hành chồng lắp
result = compare(KD_path, KetQuaPhanLoaiDat, CODE_MAP, HT_MAP)

process NN
process PNN
process TQ


In [ ]:
# 
colors = [
    "#abcee9",
    "#ffffc0",
    "#c4ff9e",
    "#ffd6a8",
    "#93ddda",
    "#1aeef7",
    "#ffa7f2",
    "#33ee33",
]
labels = ["Lúa tôm", "Lúa", "CHN", "CLN", "TS", "Sông", "Đất xây dựng", "Rừng"]

In [ ]:
save_result(result, HT_MAP)

In [ ]:
def save_result(result, HT_MAP, colors):
    cmap = ListedColormap(colors)
    save_path = "ThuanHoa/KetQua"
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    for k, v in result.items():
        rs = merge_arrays(v, nodata=np.nan)
        rs.rio.to_raster(f"{save_path}/{k}.tif")
        img = rs.plot(cmap=cmap, add_colorbar=False)
        cbar = plt.colorbar(img)
        cbar.ax.set_yticklabels(labels)
        plt.title(f'{HT_MAP[k]["name"]}')
        plt.axis("off")
        plt.show()

In [6]:
# hiển thị kết quả
xx = []

for k, v in result.items():
    rs = merge_arrays(v, nodata=np.nan)
    xx.append(rs.squeeze(drop=True))
xx = xr.concat(xx, pd.Index([HT_MAP[x]["name"] for x in HT_MAP], name="name"))

colorval = list(range(len(colors)))
options = {
    "cmap": colors,
    "clim": (0, 8),
    "aspect": "equal",
    "height": 400,
    "colorbar_opts": {
        "major_label_overrides": dict(zip(colorval, labels)),
        "major_label_text_align": "left",
        "ticker": FixedTicker(ticks=colorval),
    },
}

xx.hvplot(
    groupby="name",
    rasterize=True,  # Use Datashader, particularly useful for dask arrays
    aggregator=reductions.mode(),  # Datashader selects mode value, requires 'hv.Image'
).options(opts.Image(**options))

:DynamicMap   [name]
   :Image   [y,x]   (value)